# Library

In [1]:
import time
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.cuda import amp
from torch.utils.tensorboard import SummaryWriter
from glob import glob
from tqdm import tqdm

from libs.Trainer.Trainer import Trainer

# Config

In [2]:
class TrainGlobalConfig:
    train_batch_size: int = 256
    val_batch_size: int = 256
    n_epochs: int = 90
    lr: float = 0.1

    #img_size = 224
    num_workers = 16
        
    w_folder = 'weights/'
    l_folder = 'logs/'
    
    f_name = 'DDRNet23_slim'
    
    weight_path = w_folder + f_name 
    log_path = 'text_logs/' + f_name 
    # -------------------
    verbose = True
    verbose_step = 1
    
config = TrainGlobalConfig

In [3]:

def get_opt_and_scheduler(model, loader, lr, scheduler='step', Adam=False):
    if Adam:
        Adam_config = {"lr" : lr, "betas" : (0.9, 0.999), "eps" : 1e-08}
        optimizer = torch.optim.Adam(model.parameters(), **Adam_config)
    else:
        SGD_config = {"lr" : lr, "momentum" : 0.9, "weight_decay" : 0.0001}
        optimizer = torch.optim.SGD(model.parameters(), **SGD_config)
    if scheduler == 'onecycle':
        scheduler_params = dict(
            max_lr=lr,
            epochs=config.n_epochs,
            steps_per_epoch=int(len(loader)),
            pct_start=0.1,
            anneal_strategy='cos', 
            final_div_factor=10**5
        )
        SchedulerClass = torch.optim.lr_scheduler.OneCycleLR(optimizer, **scheduler_params)
    elif scheduler == 'step': 
        SchedulerClass = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(len(loader))*30, gamma=0.1)
    
    return optimizer, SchedulerClass


# Dataset & DataLoader

In [4]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
train_dataset = datasets.ImageFolder(
        '/home/sugimoto/datasets/imagenet/train',
        transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
validation_dataset = datasets.ImageFolder(
        '/home/sugimoto/datasets/imagenet/val',
        transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])
)

train_data_loader = DataLoader(
        train_dataset,
        batch_size = config.train_batch_size,
        shuffle = True,
        num_workers=config.num_workers,
        pin_memory=True,
)
validation_data_loader = DataLoader(
        validation_dataset,
        batch_size = config.val_batch_size,
        shuffle = False,
        num_workers=config.num_workers,
        pin_memory=True
)

In [5]:
#len(train_data_loader)

# Model

In [6]:
from libs.models.DDRNet_23_slim import get_model
from libs.models.ghost_net import ghost_net
#model = ghost_net()
#model = models.resnet18(pretrained=False)
model = get_model()
model


DualResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (relu): ReLU()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bia

# Train

In [ ]:
optimizer, scheduler = get_opt_and_scheduler(model, train_data_loader, config.lr)
device = torch.device('cuda:0')
criterion = nn.CrossEntropyLoss()

fitter = Trainer(model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion, device=device, config=config)
fitter.fit(train_data_loader, validation_data_loader)

/home/sugimoto/miniconda3/envs/sem/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


iter : 0 time : 22sec
iter : 50 time : 82sec
iter : 100 time : 72sec
iter : 150 time : 78sec
iter : 200 time : 76sec
iter : 250 time : 87sec
iter : 300 time : 84sec
iter : 350 time : 84sec
iter : 400 time : 82sec


In [ ]:
exit()